In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import email
import email.policy
from collections import Counter
from sklearn.model_selection import train_test_split

In [3]:
downloads_path = '/Users/henry.mcgowan/Downloads/'
folder = ['spam', 'easy_ham']
# 'spam_2', 'hard_ham',  'spam 2', 'hard_ham 2', 'easy_ham 2'

# get data set size 
file_names = {i: os.listdir(downloads_path + i) for i in folder}

In [122]:
# method 2
import tarfile
import os
def fetch_spam_data():
    spam_root = "http://spamassassin.apache.org/old/publiccorpus/"
    ham_url = spam_root + "20030228_easy_ham.tar.bz2"
    spam_url = spam_root + "20030228_spam.tar.bz2"

    spam_path = Path() / "datasets" / "spam"
    spam_path.mkdir(parents=True, exist_ok=True)
    for dir_name, tar_name, url in (("easy_ham", "ham", ham_url),
                                    ("spam", "spam", spam_url)):
        if not (spam_path / dir_name).is_dir():
            path = (spam_path / tar_name).with_suffix(".tar.bz2")
            print("Downloading", path)
            urllib.request.urlretrieve(url, path)
            tar_bz2_file = tarfile.open(path)
            tar_bz2_file.extractall(path=spam_path)
            tar_bz2_file.close()
    return [spam_path / dir_name for dir_name in ("easy_ham", "spam")]

In [4]:
def load_email(filepath):
    with open(filepath, "rb") as f:
        return email.parser.BytesParser(policy=email.policy.default).parse(f)
    
for i in file_names:
    print(i + ' length: ' + str(len(file_names[i])))

spam length: 501
easy_ham length: 2501


In [5]:
# load emails
ham = []
spam = []

for i in file_names['easy_ham']:
    ham.append(load_email(downloads_path + 'easy_ham' + '/' + i))

for i in file_names['spam']:
    spam.append(load_email(downloads_path + 'spam' + '/' + i))

In [123]:
# method 2
ham_dir, spam_dir = fetch_spam_data()

ham_filenames = [f for f in sorted(ham_dir.iterdir()) if len(f.name) > 20]
spam_filenames = [f for f in sorted(spam_dir.iterdir()) if len(f.name) > 20]

AttributeError: module 'os' has no attribute 'Path'

In [116]:
for i in ham[4:10]:
    print(i.get_content().strip())

At 10:34 AM -0700 on 8/28/02, Jim Whitehead wrote:


> Great, this is half of what I'd need to become Spider Man! Now all I need to
> figure out is how to do that spider web shooting thing.

...That and be able to stick yourself upside down on a 20 foot ceiling from
a standing jump...

I remember someone recently doing the calculations in kilocalories required
to be spiderman somewhere. Kind of like those flaming processor "analyses"
done a couple of years ago...

Cheers,
RAH

-- 
-----------------
R. A. Hettinga <mailto: rah@ibuc.com>
The Internet Bearer Underwriting Corporation <http://www.ibuc.com/>
44 Farquhar Street, Boston, MA 02131 USA
"... however it may deserve respect for its usefulness and antiquity,
[predicting the end of the world] has not been found agreeable to
experience." -- Edward Gibbon, 'Decline and Fall of the Roman Empire'
>Anyway I'm making good progress and my implementation works reasonably
>well.  It now seems blindingly clear that we want a tightly-integrated

In [117]:
for i in spam[0:10]:
    print(i.get_content())

Dear Homeowner,
 
Interest Rates are at their lowest point in 40 years!

We help you find the best rate for your situation by
matching your needs with hundreds of lenders!

Home Improvement, Refinance, Second Mortgage,
Home Equity Loans, and More! Even with less than
perfect credit!

This service is 100% FREE to home owners and new
home buyers without any obligation. 

Just fill out a quick, simple form and jump-start
your future plans today!


Visit http://61.145.116.186/user0201/index.asp?Afft=QM10






To unsubscribe, please visit:

http://61.145.116.186/light/watch.asp



ATTENTION: This is a MUST for ALL Computer Users!!!

*NEW-Special Package Deal!*

Norton SystemWorks 2002 Software Suite -Professional Edition-

Includes Six - Yes 6! - Feature-Packed Utilities
ALL For 1 Special LOW Price!

This Software Will:
- Protect your computer from unwanted and hazardous viruses
- Help secure your private & valuable information
- Allow you to transfer files and send e-mails safely
- Backup

KeyError: 'multipart/alternative'

In [8]:
def get_email_structure(email):
    # is the instance an email?
    if isinstance(email, str):
        return email
    
    # payload breaks a multipart email up, returns content of a single part one
    payload = email.get_payload()
    if isinstance(payload, list):
        multipart = ", ".join([get_email_structure(sub_email)
                               for sub_email in payload])
        return f"multipart({multipart})"
    else:
        return email.get_content_type()
    
def structures_counter(emails):
    structures = Counter()
    for email in emails:
        structure = get_email_structure(email)
        structures[structure] += 1
    return structures


In [9]:
structures_counter(ham).most_common()

[('text/plain', 2409),
 ('multipart(text/plain, application/pgp-signature)', 66),
 ('multipart(text/plain, text/html)', 8),
 ('multipart(text/plain, text/plain)', 4),
 ('multipart(text/plain)', 3),
 ('multipart(text/plain, application/octet-stream)', 2),
 ('multipart(text/plain, multipart(text/plain, text/plain), text/rfc822-headers)',
  1),
 ('multipart(text/plain, multipart(text/plain, text/plain), multipart(multipart(text/plain, application/x-pkcs7-signature)))',
  1),
 ('multipart(text/plain, application/x-java-applet)', 1),
 ('multipart(text/plain, text/enriched)', 1),
 ('multipart(multipart(text/plain, text/plain, text/plain), application/pgp-signature)',
  1),
 ('multipart(text/plain, multipart(text/plain))', 1),
 ('multipart(text/plain, application/ms-tnef, text/plain)', 1),
 ('multipart(text/plain, video/mng)', 1),
 ('multipart(text/plain, application/x-pkcs7-signature)', 1)]

In [10]:
structures_counter(spam).most_common()

[('text/plain', 219),
 ('text/html', 183),
 ('multipart(text/plain, text/html)', 45),
 ('multipart(text/html)', 20),
 ('multipart(text/plain)', 19),
 ('multipart(multipart(text/html))', 5),
 ('multipart(text/plain, image/jpeg)', 3),
 ('multipart(text/html, application/octet-stream)', 2),
 ('multipart(multipart(text/plain, text/html), image/gif)', 1),
 ('multipart/alternative', 1),
 ('multipart(text/plain, application/octet-stream)', 1),
 ('multipart(multipart(text/html), application/octet-stream, image/jpeg)', 1),
 ('multipart(text/html, text/plain)', 1)]

In [24]:
spam[6].get_content()

'<STYLE type="text/css">\n<!--\nP{\n  font-size : 12px;\n  color : navy;\n}\n-->\n</STYLE>\n</HEAD>\n<BODY>\n<P>(Hello,This is Chinese Traditional)<BR>\n</P>\n<DIV style="width : 376px;height : 12px;top : 195px;left : 238px;\n  position : absolute;\n  z-index : 3;\n  visibility : visible;\n  background-color : lime;\n" id="Layer3"></DIV>\n<DIV style="width : 73px;height : 14px;top : 172px;left : 238px;\n  position : absolute;\n  z-index : 2;\n  visibility : visible;\n  background-color : olive;\n" id="Layer2"></DIV>\n<DIV style="width : 39px;height : 13px;top : 149px;left : 239px;\n  position : absolute;\n  z-index : 1;\n  visibility : visible;\n  background-color : silver;\n" id="Layer1"></DIV>\n<P>��子�]件 ――21世�o最有效的宣�鞣绞剑�<BR>\n&nbsp; &nbsp;您想����地10�f人同一�r刻知道您的�V告�幔磕�想方便快捷的宣�髂�的企�I�幔�\n<BR>\n&nbsp; &nbsp;您想在商�I����中步步���紫��C�幔�\n您想一次�碛����|����在商�I客���幔�<BR>\n&nbsp; &nbsp;越�碓蕉嗟恼{查表明 ，�W路直�N����成��未���I�N方式的主流，而E-mail�⑹蔷W上�I�N最常用也是最��用的工具。<BR>\n其特�c：�����V，�r效高，�r格低。��eMarketer估��，美��

In [18]:
X = np.array(ham + spam)
y = np.array([0]* len(ham) + [1]*len(spam))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

/var/folders/lt/rzwpnmgj5v58lhpqgl3x_9zr0000gq/T/ipykernel_59024/921167800.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X = np.array(ham + spam)


In [246]:
from bs4 import BeautifulSoup as bs
import re
from html import unescape


def html_to_plain_text(html):
    text = re.sub('<head.*?>.*?</head>', '', html, flags=re.M | re.S | re.I)
    text = re.sub('<a\s.*?>', ' HYPERLINK ', text, flags=re.M | re.S | re.I)
    text = re.sub('<.*?>', '', text, flags=re.M | re.S)
    text = re.sub(r'(\s*\n)+', '\n', text, flags=re.M | re.S)
    return unescape(text)
    
    
def html_to_text_my_version(html):
    dat = BeautifulSoup(html.get_content(), 'html.parser')
    clean = dat.get_text().replace('\n', ' ').replace('\xa0', ' ').replace('\t', ' ').replace('\\ ', ' ')
    clean_links = re.sub('(http|ftp|https):\/\/([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:\/~+#-]*[\w@?^=%&\/~+#-])',
                         ' HYPERLINK ', clean, flags = re.M | re.S)
    return clean_links

In [235]:
# test parser for functionality - looks pretty good, we can move on to parsing the strings now
html_emails = [i for i in X_train[y_train == 1] if get_email_structure(i) == 'text/html']
#clean_emails = html_to_text_my_version(html_emails[33])
#html_to_text_my_version(html_emails[10])
html_to_text_my_version(html_emails[10])


"  Do you like Sexy Animals doing the wild thing? We have the super hot content on the Internet! This is the site you have heard about. Rated the number one adult site three years in a row! - Thousands of pics from hardcore fucking, and cum shots to pet on girl.  - Thousands videos  So what are you waiting for?  CLICK HERE   YOU MUST BE AT LEAST 18 TO ENTER!       You have received this advertisement because you have opted in  to receive  free adult internet offers and   specials through our affiliated websites. If you do not wish to receive  further emails or have received the   email in error you may opt-out of our database by clicking here:  CLICK HERE Please allow 24hours for removal. This e-mail is sent in compliance with the Information Exchange Promotion and Privacy Protection Act.   section 50 marked as 'Advertisement' with valid 'removal' instruction.     [NKIYs5]   "

In [278]:
# how to extract information out of the strings?
# im skipping a step - all of the emails need to be preprocessed. need to address multipart

def clean_emails(email):
    structure = get_email_structure(email)
    if structure in ['text/html', 'text/plain']:
        clean = html_to_text_my_version(email)
        return clean
    elif isinstance(structure, tuple):
        parts = email.get_payload()
        parsed = ' '.join([clean_emails(x) for x in parts])
        clean_multi = ' '.join(parsed)
        return clean_multi
    
        
        

In [293]:
# stalling on the encoding issue - going to stop here and pick this up tomorrow.
# will do linear algebra in the AM before coming back to this! 
[clean_emails(x) for x in X_train]

LookupError: unknown encoding: DEFAULT

In [ ]:
# next up is figuring out how to extract information from the text. first, we'll need to stem to minimize noise
# in the data and set us up for 